The Environment: A Simple GridWorld
Let's create a 3x4 grid. The agent's goal is to get from a starting position to one of the two terminal states.

Grid: 3 rows, 4 columns.

States: 12 states (0-11).

Actions: 0: up, 1: down, 2: left, 3: right.

Start State: State 8 (bottom-left).

Terminal States:

State 3 (top-right) with a reward of +10.

State 7 (middle-right) with a reward of -10 (a cliff).

Walls: State 5 is an obstacle the agent cannot enter.

Reward: Every non-terminal step gives a small negative reward of -0.1 to encourage finding the goal faster.

In [ ]:
import numpy as np
import random
from collections import defaultdict

class GridWorld:
    """
    A simple GridWorld environment for the agent.
    This class provides the mechanics of the world (how to move, what are the rewards),
    but it does NOT expose the transition probabilities P(s'|s,a).
    The agent must learn them through experience.
    """
    def __init__(self):
        # Define grid dimensions
        self.rows = 3
        self.cols = 4
        self.num_states = self.rows * self.cols
        self.num_actions = 4  # 0: up, 1: down, 2: left, 3: right

        # Define special states
        self.start_state = 8
        self.goal_state = 3
        self.trap_state = 7
        self.wall_state = 5

        self.terminal_states = [self.goal_state, self.trap_state]

    def step(self, state, action):
        """
        Perform an action in a given state.
        Returns: (next_state, reward, done)
        """
        if state in self.terminal_states:
            return state, 0, True

        row, col = state // self.cols, state % self.cols

        # Calculate next position based on action
        if action == 0:  # up
            row = max(row - 1, 0)
        elif action == 1:  # down
            row = min(row + 1, self.rows - 1)
        elif action == 2:  # left
            col = max(col - 1, 0)
        elif action == 3:  # right
            col = min(col + 1, self.cols - 1)

        next_state = row * self.cols + col

        # Agent cannot enter the wall, so it stays in place
        if next_state == self.wall_state:
            next_state = state

        # Define rewards
        if next_state == self.goal_state:
            reward = 10
        elif next_state == self.trap_state:
            reward = -10
        else:
            reward = -0.1 # Small cost for each move

        done = next_state in self.terminal_states
        return next_state, reward, done

**MC Basic Algorithm Implementation**

This is a direct, though inefficient, implementation of the pseudocode. It iterates through every state-action pair and generates many episodes starting from that pair to evaluate its Q-value.

In [ ]:
def mc_basic(env, num_iterations, episodes_per_pair, gamma=0.99):

    # Initialize Q-values and returns
    policy = {s: random.randint(0, env.num_actions - 1) for s in range(env.num_states)}
    Q = defaultdict(float) # Q[s][a] = 0.0

    # Main Loop for k-th iteration
    for k in range(num_iterations):
        print(f'--- Iteration {k + 1}/{num_iterations} ---')

        # --- Policy evaluation ---
        for s in range(env.num_states):
            if s in env.terminal_states or s == env.wall_state:
                continue
            for a in range(env.num_actions):
                returns_for_pair = []
                for _ in range(episodes_per_pair):
                    episode_return = 0
                    t = 0

                    current_state, reward, done = env.step(s, a)
                    episode_return += reward

                    # Following the policy pi_k for the rest of the episode
                    while not done:
                        action_from_policy = policy[current_state]
                        next_state, reward, done = env.step(current_state, action_from_policy)
                        episode_return += (gamma ** (t+1)) * reward
                        current_state = next_state
                        t += 1
                    returns_for_pair.append(episode_return)

                # q_pi_k(s, a) = average of returns for (s, a)
                if returns_for_pair:
                    Q[(s, a)] = np.mean(returns_for_pair)

        # --- Policy improvement ---
        for s in range(env.num_states):
            if s in env.terminal_states or s == env.wall_state:
                continue

            # Find the best action for state s
            q_values_for_s = [Q[(s, a)] for a in range(env.num_actions)]

        best_action = np.argmax(q_values_for_s)
        # Update policy
        policy[s] = best_action

    print('--- MC Basic Finished ---')
    return policy, Q

In [ ]:
mc_basic(env=GridWorld(), num_iterations=50, episodes_per_pair=50, gamma=0.99)

**MC Exploring Starts Implementation**

This version is much more efficient. It generates one episode at a time (with a random starting pair) and updates the Q-values and policy for all state-action pairs visited in that episode.

In [ ]:
def mc_exploring_starts(env, num_episodes, gamma=0.99, max_steps_per_episode=5):

    # Initialize Q-values and returns
    policy = {s: 0 for s in range(env.num_states)}
    Q = defaultdict(float)  # Q[s][a] = 0.0
    Returns = defaultdict(float)  # Returns[s][a] = []
    Num = defaultdict(int)  # Num[s][a] = 0

    # For each episode
    for i in range(num_episodes):
        if (i + 1) % 1000 == 0:
            print(f'--- Episode {i + 1}/{num_episodes} ---')

        # --- Episode Generation ---
        # Randomly select a starting state and action
        s0 = random.randint(0, env.num_states - 1)
        while s0 in env.terminal_states or s0 == env.wall_state:
            s0 = random.randint(0, env.num_states - 1)
        a0 = random.randint(0, env.num_actions - 1)

        current_state = s0
        current_action = a0

        episode = []
        done = False
        steps = 0

        # Generate the episode of length T
        while not done and steps < max_steps_per_episode:
            next_state, reward, done = env.step(current_state, current_action)
            episode.append((current_state, current_action, reward))
            current_state = next_state

            if not done:
                current_action = policy[current_state]  # Follow the policy for the next action

        # --- Policy Evaluation and Episode Improvement for Each Step ---
        g = 0  # Initialize g, the return
        visited_pairs = set()

        # For each step of the episode, t = T-1, T-2, ..., 0
        for t in range(len(episode) -1, -1, -1):
            s_t, a_t, r_t_plus_1 = episode[t]
            g = gamma * g + r_t_plus_1

            # We use First Visit MC
            if (s_t, a_t) not in visited_pairs:
                visited_pairs.add((s_t, a_t))

                Returns[(s_t, a_t)] += g
                Num[(s_t, a_t)] += 1

                Q[(s_t, a_t)] = Returns[(s_t, a_t)] / Num[(s_t, a_t)]

                q_values_for_s_t = [Q[(s_t, a)] for a in range(env.num_actions)]
                policy[s_t] = np.argmax(q_values_for_s_t)

    print('--- MC Exploring Starts Finished ---')
    return policy, Q

In [ ]:
mc_exploring_starts(env=GridWorld(), num_episodes=5, gamma=0.99, max_steps_per_episode=5)

**MC ε-Greedy Implementation**

This is the most practical of the three. It removes the need for exploring starts by using an ε-greedy policy, which ensures continuous exploration.

In [ ]:
 def mc_epsilon_greedy(env, num_episodes, gamma=0.99, epsilon=0.1):

    # Initialize Q-values and returns
    Q = defaultdict(float)
    Returns = defaultdict(float)
    Num = defaultdict(int)
    policy = {s: np.ones (env.num_actions) / env.num_actions for s in range(env.num_states)}

    # For each episode, do:
    for i in range(num_episodes):
        #if (i + 1) % 1000 == 0:
        #    print(f'--- Episode {i + 1}/{num_episodes} ---')

        # --- Episode Generation ---
        current_state = env.start_state
        episode = []
        done = False

        while not done:
            # Choose action using ε-greedy policy
            action = np.random.choice(np.arange(env.num_actions), p = policy[current_state])

            next_state, reward, done = env.step(current_state, action)
            episode.append((current_state, action, reward))
            current_state = next_state

        # --- Update Q and Policy for each first-visit pairin the episode ---
        g = 0
        visited_pairs = set()

        # For each step of the episode, t = T-1, T-2, ..., 0
        for t in range(len(episode) - 1, -1, -1):
            s_t, a_t, r_t_plus_1 = episode[t]
            g = gamma * g + r_t_plus_1

            if (s_t, a_t) not in visited_pairs:
                visited_pairs.add((s_t, a_t))

                Returns[(s_t, a_t)] += g
                Num[(s_t, a_t)] += 1
                Q[(s_t, a_t)] = Returns[(s_t, a_t)] / Num[(s_t, a_t)]

                # Policy Improvement for state s_t
                best_action = np.argmax([Q[(s_t, a)] for a in range(env.num_actions)])
                # Update policy probabilities for state s_t
                for a in range(env.num_actions):
                    if a == best_action:
                        policy[(s_t, a)] = 1 - epsilon + (epsilon / env.num_actions)
                    else:
                        policy[(s_t, a)] = epsilon / env.num_actions

    print('--- MC ε-Greedy Finished ---')
    return policy, Q

